In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import kpss
import itertools
from datetime import timedelta
from pmdarima.arima.utils import nsdiffs
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
from statsmodels.tools.sm_exceptions import InterpolationWarning

warnings.filterwarnings('ignore', category=InterpolationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
# Create a date range for one year (e.g., 2020)
dates = pd.date_range(start='2020-01-01', end='2020-12-31', freq='D')

# Define units and shifts (each combination will represent a separate series)
units = ['Unit1', 'Unit2']
shifts = ['Morning', 'Evening']

data_list = []

# Loop through each combination and each date
for unit in units:
    for shift in shifts:
        # Define a different trend or base sales for each unit/shift if desired
        # Here, we use a simple example where trend intensity varies by combination.
        if unit == 'Unit1' and shift == 'Morning':
            base, trend_factor = 100, 0.1
        elif unit == 'Unit1' and shift == 'Evening':
            base, trend_factor = 120, 0
        elif unit == 'Unit2' and shift == 'Morning':
            base, trend_factor = 90, 0.15
        else:  # Unit2, Evening
            base, trend_factor = 110, 0.25
        
        for date in dates:
            weekday = date.day_name()
            # Randomly designate a holiday (e.g., 10% chance)
            holiday = np.random.choice([0, 1], p=[0.9, 0.1])
            # Generate a synthetic sales value: base + trend (based on day of year) + random noise
            day_of_year = date.timetuple().tm_yday
            noise = np.random.normal(loc=0, scale=10)
            sales = base + trend_factor * day_of_year + noise
            
            data_list.append({
                'Date': date,
                'Unit': unit,
                'Shift': shift,
                'WeekdayName': weekday,
                'HolidayYN': holiday,
                'Sales': sales
            })

# Create the DataFrame
df = pd.DataFrame(data_list)

# Optional: Display the first few rows to verify the data
print(df.head())

In [ ]:
# plot the series

# Group by Unit and Shift
groups = df.groupby(['Unit', 'Shift'])

# Loop through each group and plot the series
for (unit, shift), group in groups:
    plt.figure(figsize=(5, 3))
    plt.plot(group['Date'], group['Sales'], linestyle='-', label=f'{unit} {shift}')
    plt.title(f'Sales Series for {unit} - {shift}')
    plt.xlabel('Date')
    plt.ylabel('Sales')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# create dummies
df = pd.get_dummies(df, columns = ['WeekdayName', 'HolidayYN'], drop_first=True)

In [ ]:
# Define a function to test stationarity and seasonal differencing needs
def check_stationarity(series, kpss_alpha=0.05, m=7):
    # KPSS test for stationarity (null hypothesis: series is stationary)
    try:
        stat, p_value, lags, crit = kpss(series, regression='c', nlags='auto')
    except Exception:
        # In case of error (e.g., very short series), assume stationarity
        p_value = 1.0
    difference = 1 if p_value < kpss_alpha else 0

    # Seasonal differencing test using nsdiffs (based on Canova-Hansen)
    seasonal_order = nsdiffs(series, m=m, test='ch')
    seasonal_difference = 1 if seasonal_order > 0 else 0

    return difference, seasonal_difference

# Assuming df is your DataFrame with columns:
# 'Date', 'Unit', 'Shift', 'WeekdayName', 'HolidayYN', 'Sales'
results = []

# Group the data by Unit and Shift
for (unit, shift), group in df.groupby(['Unit', 'Shift']):
    # Ensure the series is sorted by Date
    group_sorted = group.sort_values('Date')
    sales_series = group_sorted['Sales']
    
    # Check for unit root and seasonal unit root
    diff, seas_diff = check_stationarity(sales_series)
    
    results.append({
        'Unit': unit,
        'Shift': shift,
        'difference': diff,
        'seasonal_difference': seas_diff
    })

# Create the resulting DataFrame
stationary_df = pd.DataFrame(results)

In [ ]:
# -------------------------------------------------------------
# Function to determine the non-seasonal differencing order (d)
# using the KPSS test. We increase d until the series becomes stationary.
# -------------------------------------------------------------
def determine_differencing_order(series, alpha=0.05, max_d=2):
    d = 0
    series_to_test = series.copy()
    while d <= max_d:
        try:
            statistic, p_value, n_lags, crit = kpss(series_to_test, regression='c', nlags="auto")
        except Exception as e:
            print(f"KPSS test error: {e}")
            break
        # If p_value > alpha, the series is stationary
        if p_value > alpha:
            break
        else:
            d += 1
            series_to_test = series_to_test.diff().dropna()
    return d

# -------------------------------------------------------------
# Function to determine the seasonal differencing order (D)
# using the nsdiffs function from pmdarima.
# -------------------------------------------------------------
def determine_seasonal_differencing_order(series, seasonal_period=7, max_D=1):
    D = nsdiffs(series, m=seasonal_period, max_D=max_D)
    return D

# -------------------------------------------------------------
# Function to create the difference dataframe.
# For each Unit/Shift series, calculate d and D.
# -------------------------------------------------------------
def create_difference_dataframe(df, seasonal_period=7):
    rows = []
    for (unit, shift), group in df.groupby(['Unit', 'Shift']):
        group = group.sort_values('Date')
        series = group['Sales'].astype(float)
        d = determine_differencing_order(series)
        D = determine_seasonal_differencing_order(series, seasonal_period)
        rows.append({
            'Unit': unit,
            'Shift': shift,
            'Difference': d,
            'SeasonalDifference': D
        })
    difference_df = pd.DataFrame(rows)
    return difference_df

# -------------------------------------------------------------
# Function to create the model parameters dataframe.
# It uses candidate lists for p, q, P, Q and the computed d and D values.
# -------------------------------------------------------------
def create_model_parameters_dataframe(difference_df, 
                                        p_values=[0,1,2], 
                                        q_values=[0,1,2], 
                                        P_values=[0,1], 
                                        Q_values=[0,1], 
                                        seasonal_period=7):
    rows = []
    for _, row in difference_df.iterrows():
        unit = row['Unit']
        shift = row['Shift']
        d = row['Difference']
        D = row['SeasonalDifference']
        model_number = 0
        # Generate all combinations of (p, q, P, Q)
        for p, q, P, Q in itertools.product(p_values, q_values, P_values, Q_values):
            model_number += 1
            # Build the SARIMAX parameter tuple:
            # non-seasonal order: (p, d, q)
            # seasonal order: (P, D, Q, seasonal_period)
            model_params = {
                'order': (p, d, q),
                'seasonal_order': (P, D, Q, seasonal_period)
            }
            rows.append({
                'Unit': unit,
                'Shift': shift,
                'ModelNumber': model_number,
                'ModelParameters': model_params
            })
    model_parameters_df = pd.DataFrame(rows)
    return model_parameters_df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
    
# 1. Create the difference dataframe
difference_df = create_difference_dataframe(df, seasonal_period=7)
    
# 2. Create the model parameters dataframe
model_parameters_df = create_model_parameters_dataframe(difference_df, p_values=[0,1], q_values=[0,1], 
                                                        P_values=[0], Q_values=[0], seasonal_period=7)

In [ ]:

def create_testing_date_ranges_dataframe(df, forecast_lengths=[7, 90], periods=3, shift_days=11):
    """
    Create a DataFrame with testing date ranges for each unit/shift series.
    
    For each (Unit, Shift) group in the DataFrame:
      - For each forecast_length (e.g., 7 or 90 days), generate a fixed number of periods (default 3).
      - The first testing period ends on the maximum date in the series.
      - Subsequent periods are obtained by shifting the end date to the left by shift_days (11 days).
      - The start date is computed as end date - forecast_length + 1.
      
    Parameters:
      df: pandas DataFrame containing at least the 'Date', 'Unit', and 'Shift' columns.
      forecast_lengths: list of forecast lengths (in days) to create testing periods for.
      periods: number of testing periods to generate for each forecast length.
      shift_days: number of days to shift the testing period end date for each subsequent period.
    
    Returns:
      A pandas DataFrame with columns: Unit, Shift, TestingPeriodNumber, ForecastLength, StartDate, EndDate.
    """
    rows = []
    for (unit, shift), group in df.groupby(['Unit', 'Shift']):
        group = group.sort_values('Date')
        max_date = group['Date'].max()
        for forecast_length in forecast_lengths:
            for period in range(1, periods + 1):
                # Compute shift offset: period 1 uses 0-day offset, period 2 uses shift_days, etc.
                shift_offset = (period - 1) * shift_days
                period_end = max_date - pd.Timedelta(days=shift_offset)
                period_start = period_end - pd.Timedelta(days=forecast_length - 1)
                rows.append({
                    'Unit': unit,
                    'Shift': shift,
                    'TestingPeriodNumber': f"{forecast_length}_period_{period}",
                    'ForecastLength': forecast_length,
                    'StartDate': period_start,
                    'EndDate': period_end
                })
    testing_ranges_df = pd.DataFrame(rows)
    return testing_ranges_df

In [ ]:
testing_ranges_df = create_testing_date_ranges_dataframe(df)

In [ ]:
# -------------------------------------------------------------
# Utility function to compute error metrics.
# -------------------------------------------------------------
def compute_error_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # Avoid division by zero in MAPE calculation:
    with np.errstate(divide='ignore', invalid='ignore'):
        mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true == 0, 1, y_true))) * 100
    return mae, mape, rmse

# -------------------------------------------------------------
# Function to run model testing and collect the results.
#
# For each (Unit, Shift) group:
#  1. For each testing period (defined by forecast_start and forecast_end)
#     - Use all data before forecast_start as the training set.
#     - Extract actual sales for the testing period.
#  2. For each candidate model (from the model_parameters_df):
#     - Fit a SARIMAX model on the training set.
#     - Forecast for forecast_length days.
#     - Compute MAE, MAPE, and RMSE comparing the forecast to the actual test data.
#  3. Append the results to a list.
#
# Returns a DataFrame with columns:
#   Unit, Shift, ForecastLength, ForecastStart, ForecastEnd,
#   ModelParameters (string), MAE, MAPE, RMSE
# -------------------------------------------------------------
def run_model_testing(df, testing_date_ranges_df, model_parameters_df):
    results = []
    
    # Loop over each testing period from our testing ranges dataframe.
    for _, test_row in testing_date_ranges_df.iterrows():
        unit = test_row['Unit']
        shift = test_row['Shift']
        forecast_start = test_row['StartDate']
        forecast_end = test_row['EndDate']
        forecast_length = test_row['ForecastLength']
        
        # Extract and sort data for the current unit and shift.
        group_data = df[(df['Unit'] == unit) & (df['Shift'] == shift)].sort_values('Date')
        # Set Date as the index and enforce a daily frequency
        group_data = group_data.set_index('Date').asfreq('D')
        
        # Use all available data before the forecast start date as training.
        train_data = group_data[group_data.index < forecast_start]
        # Extract test data for the forecast period.
        test_data = group_data[(group_data.index >= forecast_start) & (group_data.index <= forecast_end)]
        
        # Ensure we have a complete test period.
        if len(test_data) < forecast_length or train_data.empty:
            print(f"Skipping Unit {unit} Shift {shift} period starting {forecast_start} due to insufficient data.")
            continue

        # Get candidate models for the current unit/shift.
        models_df = model_parameters_df[(model_parameters_df['Unit'] == unit) & (model_parameters_df['Shift'] == shift)]
        
        for _, model_row in models_df.iterrows():
            model_params = model_row['ModelParameters']
            order = model_params['order']
            seasonal_order = model_params['seasonal_order']
            
            try:
                # Fit SARIMAX on training data.
                model = SARIMAX(train_data['Sales'], 
                                order=order, 
                                seasonal_order=seasonal_order,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
                model_fit = model.fit(disp=False, maxiter=200, method='lbfgs')
                
                # Forecast the specified number of steps.
                forecast = model_fit.forecast(steps=forecast_length)
                
                # Ensure forecast and test data lengths match.
                if len(forecast) != forecast_length:
                    print(f"Forecast length mismatch for Unit {unit} Shift {shift} with model {model_params}")
                    continue
                
                # Calculate error metrics.
                mae, mape, rmse = compute_error_metrics(test_data['Sales'].values[:forecast_length], forecast.values)
                
                # Format the model parameters as a string (e.g., "(1,0,0)x(1,0,0,7)")
                model_param_str = f"({order[0]},{order[1]},{order[2]})x({seasonal_order[0]},{seasonal_order[1]},{seasonal_order[2]},{seasonal_order[3]})"
                
                results.append({
                    'Unit': unit,
                    'Shift': shift,
                    'ForecastLength': forecast_length,
                    'ForecastStart': forecast_start,
                    'ForecastEnd': forecast_end,
                    'ModelParameters': model_param_str,
                    'MAE': mae,
                    'MAPE': mape,
                    'RMSE': rmse
                })
            except Exception as e:
                # In case the model fails to fit, log and continue.
                print(f"Error for Unit {unit} Shift {shift} with model {model_params}: {e}")
                continue
    
    # Convert results list into a DataFrame.
    results_df = pd.DataFrame(results)
    return results_df

In [ ]:
model_testing_results = run_model_testing(df, testing_ranges_df, model_parameters_df)

In [ ]:
model_testing_results.head()